In [159]:
import os
import re
import csv
from itertools import zip_longest

resultsFolder = 'experimental_results'

resList = [['test-case', 'cfg', 'date', 'total_time-time', 
            'total_time-main', 'beta', 'avg-PIs', 'n-PIs', 
            '1st-PI', '2nd-PI', 'avg(2-6)', 'avg(2-11)']]

if resultsFolder not in os.listdir():
    print(f"ERROR: {resultsFolder} folder not found")
    exit(1)
    
# for each test case
for tc in os.listdir(resultsFolder):
    clusterConf = os.listdir(f"{resultsFolder}/{tc}")
    
    # for each cluster configuration
    for cc in clusterConf:
        testDates = os.listdir(f"{resultsFolder}/{tc}/{cc}")
        
        # for each date
        for td in testDates:
            timeOutFile = open(f"{resultsFolder}/{tc}/{cc}/{td}/time.out", "r")
            lmpOutFile  = open(f"{resultsFolder}/{tc}/{cc}/{td}/lmp.out", "r")
            
            # time command outputs the elapsed walltime 
            # in the last two words: ... 10.300 total
            # split()[-2] gets the time value
            wallTime = float(timeOutFile.read().split()[-2])
            
            '''
            Parse MO833 profiling info
            '''
            PIavg    = {} # PIavg[0]  = x means process 0 showed PIavg = x
            beta     = {} # beta[0]   = x means process 0 showed beta = x
            parIter  = {} # parIter[x,y] = z is duration of PI y in process x

            for line in lmpOutFile.readlines():
                # remove line break
                line = line.replace('\n','')
                
                if "[MO833] Paramount Iteration" in line:
                    procNum    = float(line.split(',')[1])
                    PInum      = float(line.split(',')[2])
                    duration   = float(line.split(',')[3])
                    parIter[procNum,PInum] = duration
               
                elif "[MO833] Beta" in line:
                    procNum = float(line.split(',')[1])
                    betaVal = float(line.split(',')[2])
                    beta[procNum] = betaVal

                elif "[MO833] PI avg" in line:
                    procNum  = float(line.split(',')[1])
                    PIavgVal = float(line.split(',')[2])
                    PIavg[procNum] = PIavgVal
                
                elif "[MO833] Total time" in line:
                    mainTime = float(line.split(',')[1])

            # close files
            timeOutFile.close()
            lmpOutFile.close()
                    
            nProcs = len(beta)
            nPI    = len([z for ((x,y),z) in parIter.items() if x == 0])

            
            '''
            Print some useful infermation about quality of the data
            '''
            betaDiff  = max(beta.values())-min(beta.values())
            PIavgDiff = max(PIavg.values())-min(PIavg.values())
            timeDiff  = wallTime-mainTime

            print(f"MaxBeta-MinBeta  ={betaDiff}")
            print(f"MaxPIavg-MinPIavg={PIavgDiff}s")
            print(f"MainTime-WallTime={timeDiff}s")
            
            # average duration for par iter from 2 to 6 @ proc 0
            aux = [dur for ((proc,PI),dur) in parIter.items()
                   if proc == 0 and 2 <= PI and PI <= 6]
            avg_2_6  = sum(aux)/len(aux)
            
            # average duration for par iter from 2 to 11 @ proc 0
            aux = [dur for ((proc,PI),dur) in parIter.items()
                   if proc == 0 and 2 <= PI and PI <= 11]
            avg_2_11 = sum(aux)/len(aux)
            
            resList.append([
                tc, #test case
                cc, #cluster configuration
                td, #test dates
                wallTime, #time measured with time
                mainTime, #main function time
                beta[0],
                PIavg[0],
                nPI,
                parIter[0,1], #duration of 1st par it @ proc 0
                parIter[0,2],
                avg_2_6,
                avg_2_11,
            ])

with open('experimental_results.summary.csv', 'w', newline='') as csvSumary:
    wr = csv.writer(csvSumary)
    wr.writerows(resList)
            

MaxBeta-MinBeta  =2.9000000000001247e-05
MaxPIavg-MinPIavg=0.0s
MainTime-WallTime=0.08297799999999889s


In [161]:
import matplotlib as plt

